<a href="https://colab.research.google.com/github/SiphiweAI/IsAI/blob/main/IsAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope
- Use pretrained models that will review video, text or and/or image content and classify as being ai generated or not
## Inputs
- For Text - Hugging Face Transformers
- For Images -
- For Videos -
## Outputs
- Three separate models for text, image, and video detection, each trained and optimized independently.

- A wrapper API or service that routes incoming data to the appropriate model based on content type.

- Unified output format that reports whether the input is AI-generated, with modality-specific confidence scores.

# Text Model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [2]:
models_info = [
    {"name": "roberta-base-openai-detector", "sigmoid": True},
    {"name": "prasoonmhwr/ai_detection_model", "sigmoid": True},
    {"name": "openai-community/roberta-large-openai-detector", "sigmoid": True},
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
models = []

In [4]:
for m in models_info:
    tok = AutoTokenizer.from_pretrained(m["name"])
    mdl = AutoModelForSequenceClassification.from_pretrained(m["name"]).to(device)
    mdl.eval()
    models.append({
        "tokenizer": tok,
        "model": mdl,
        "sigmoid": m["sigmoid"]
    })

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at openai-community/roberta-large-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def predict_ave(text: str) -> float:
    """
    Predict average probability that the input text is AI-generated
    by averaging outputs from multiple models.

    Args:
        text (str): Input text to classify.

    Returns:
        float: Average AI-generated probability (0 to 1).
    """
    probs = []
    with torch.no_grad():
        for entry in models:
            tokenizer = entry["tokenizer"]
            model = entry["model"]
            sigmoid_flag = entry["sigmoid"]

            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}

            outputs = model(**inputs)
            logits = outputs.logits

            if sigmoid_flag:
                prob = torch.sigmoid(logits)[0][0].item()
            else:
                # If model outputs logits for two classes, apply softmax and take AI-generated class probability
                prob = F.softmax(logits, dim=1)[0][1].item()

            probs.append(prob)

    avg_prob = sum(probs) / len(probs)
    return avg_prob

In [6]:
text = "In this paper, we explore..."
result = predict_ave(text)
print(result)

0.445468803246816


# Image Model

# Videos Model